In [3]:
from database import Database

db = Database()
name, id, url, total_reviews, address = ['Absee Restaurant', 'JIDFHK7889jdsl', 'https:///places.google.com/places_api?restaurant&other_stuff', 232, '123 Some Street, Dark Ally']
sql = u
db.update_rows(sql, *(name, id, url,  total_reviews, address))

Connection opened successfully.


In [ ]:
from Yelp_business_search import get_yelp_places_by_location

get_yelp_places_by_location()

In [6]:
list1 = [1,2,3]
list2 = [2,3,4]


1 2
2 3
3 4


In [1]:
from review_scraper import scrape_yelp_reviews
scrape_yelp_reviews("8mmM20X369_Do04kW7-TTA", "https://www.yelp.com/biz/holy-cow-new-york-4?adjust_creative=xl_eXoprFhB3R9K5U7OxqQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=xl_eXoprFhB3R9K5U7OxqQ")

Connection opened successfully.


In [1]:
from review_scraper import scrape_google_reviews
scrape_google_reviews("57c3c1616c90aaafb6ab54f6f36c27e303da9ad0", "https://maps.google.com/?cid=4988939335835406517")

ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=83.0.4103.61)
